In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import cv2
import numpy as np
import os
import shutil
import keras
from scipy.spatial import distance as dist
from tensorflow.keras.models import Sequential, Model, load_model
from google.colab.patches import cv2_imshow
from itertools import combinations
import math

In [ ]:
faceDetector = cv2.CascadeClassifier('/content/gdrive/MyDrive/ProjectDeliveries/faceMask_SocialDistance/haarcascade_frontalface_alt2.xml')
faceMaskModel = load_model("/content/gdrive/MyDrive/ProjectDeliveries/faceMask_SocialDistance/FaceMaskModel.model")


In [ ]:
def MaskDetector(img):
  preds = faceMaskModel.predict(np.expand_dims(img, axis=0))[0]
  if round(preds[0]) == 0:
      return 1
  else:
      return 0

In [ ]:
def calculateDistance(x1,y1,x2,y2):
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return dist

In [ ]:
MODEL_PATH = "/content/gdrive/MyDrive/ProjectDeliveries/faceMask_SocialDistance/yolo"
labelsPath = os.path.sep.join([MODEL_PATH, "coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")
weightsPath = os.path.sep.join([MODEL_PATH, "yolov3.weights"])
configPath = os.path.sep.join([MODEL_PATH, "yolov3.cfg"])
net = cv2.dnn.readNet(weightsPath, configPath)
classes = []
with open(labelsPath, "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [ ]:
def detectSocialDistance(img):
    height, width, channels = img.shape
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    center_points = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
 
            if confidence > 0.5 and class_id == 0:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h, center_x, center_y])
                center_points.append([center_x, center_y])
                confidences.append(float(confidence))
                class_ids.append(class_id)
 
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
 
    combination_points = list(combinations(center_points, 2))
 
    font = cv2.FONT_HERSHEY_DUPLEX
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h, box_center_x, box_center_y = boxes[i]
 
            color = colors[i]
            cv2.circle(img, (center_x, center_y), 3, (0, 0, 255), cv2.FILLED)
 
            for points in combination_points:
                center_x, center_y = points[0]
                prev_center_x, prev_center_y = points[1]
                euclidean_distance = calculateDistance(center_x, center_y, prev_center_x, prev_center_y)
 
                width_of_3_tiles = 335
 
                if euclidean_distance < width_of_3_tiles and euclidean_distance > 150:
                    if box_center_x == center_x or box_center_y == center_y:
                        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
                if euclidean_distance > width_of_3_tiles and euclidean_distance > 150:
                    if box_center_x == center_x or box_center_y == center_y:
                        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    return img    

In [ ]:
# image_path = "/content/gdrive/MyDrive/ProjectDeliveries/faceMask_SocialDistance/testImageNoMaskNoDistance.png"
image_path = "/content/gdrive/MyDrive/ProjectDeliveries/faceMask_SocialDistance/testImageMaskNoDistance.jpeg"
# image_path = "/content/gdrive/MyDrive/ProjectDeliveries/faceMask_SocialDistance/testImageMaskDistance2.jpeg"


img = cv2.imread(image_path)
cv2_imshow(img)
grayscaled_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


face_coordinates = faceDetector.detectMultiScale(grayscaled_img)
for (x, y, w, h) in face_coordinates:
  image = img[y:y + h, x:x + w]
  image = cv2.resize(image, (224, 224))
  result = MaskDetector(image)
  if result == 1:
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
  else:
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
# img = cv2.resize(img, (700, 700))
img = detectSocialDistance(img)
cv2_imshow(img)

Output hidden; open in https://colab.research.google.com to view.